In [1]:
'''
Sanjay Singh
san.singhsanjay@gmail.com
April-2021
To train a neural network for Image Captioning - on Google Colab
This script file is copy of script_training.py, only paths have changed in this file to make it to work on Google Colab
'''

'\nSanjay Singh\nsan.singhsanjay@gmail.com\nApril-2021\nTo train a neural network for Image Captioning - on Google Colab\nThis script file is copy of script_training.py, only paths have changed in this file to make it to work on Google Colab\n'

In [2]:
# start by connecting gdrive into the google colab
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
# packages
import numpy as np
from numpy import array
from numpy import savetxt
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [4]:
# constants
EPOCHS = 20
NUMBER_PICS_PER_BATCH = 3
EMBEDDING_DIM = 200

In [5]:
# function to update status
def percentage_progress(completed, total):
	perc_progress = (completed / total) * 100
	perc_progress = round(perc_progress, 2)
	return perc_progress

In [6]:
# data generator function
'''
To run parts of this function in ipython without calling function
image_name_captions = train_image_caption
image_features = train_image_feature
num_pics_per_batch = NUMBER_PICS_PER_BATCH
'''
def data_generator(image_name_captions, image_features, wordtoix, max_caption_length, vocab_size, num_pics_per_batch):
	X1, X2, y = list(), list(), list()
	n = 0
	while(True):
		for i in range(image_name_captions.shape[0]):
			n += 1
			image_name = image_name_captions.iloc[i]['image']
			image_feature = image_features[image_name]
			captions = image_name_captions.iloc[i]['caption']
			captions_list = captions.split("#")
			for caption in captions_list:
				seq = [wordtoix[word] for word in caption.split(' ') if word in wordtoix]
				for j in range(len(seq)):
					in_seq, out_seq = seq[:j], seq[j]
					in_seq = pad_sequences([in_seq], maxlen=max_caption_length, padding='post')[0]
					out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
					X1.append(image_feature)
					X2.append(in_seq)
					y.append(out_seq)
			if(n == num_pics_per_batch):
				yield [array(X1), array(X2)], array(y)
				X1, X2, y = list(), list(), list()
				n = 0

In [7]:
# paths
train_image_caption_path = "/content/gdrive/MyDrive/Flickr8k_ImageCaptioning/processed_data/train_image_caption_processed.csv"
train_image_bottleneck_feature_path = "/content/gdrive/MyDrive/Flickr8k_ImageCaptioning/processed_data/train_imagename_bottleneck_feat.csv"
vocabulary_path = "/content/gdrive/MyDrive/Flickr8k_ImageCaptioning/processed_data/vocabulary.txt"
glove_model_path = "/content/gdrive/MyDrive/Flickr8k_ImageCaptioning/pre-trained_models/glove.6B.200d.txt"
max_caption_length_path = "/content/gdrive/MyDrive/Flickr8k_ImageCaptioning/processed_data/max_caption_length.txt"
target_path = "/content/gdrive/MyDrive/Flickr8k_ImageCaptioning/output/trained_models/"

In [8]:
# reading vocabulary
vocabulary = list()
f_ptr = open(vocabulary_path, 'r')
lines = f_ptr.readlines()
for line in lines:
	vocabulary.append(line.strip())
print("Completed reading vocabulary file")

Completed reading vocabulary file


In [9]:
# creating word-to-index and index-to word dictionary
wordtoix = dict()
ixtoword = dict()
#ix = 1
for i in range(len(vocabulary)):
	wordtoix[vocabulary[i]] = i
	ixtoword[i] = vocabulary[i]
	#ix += 1
print("Created word-to-index and index-to-word dictionary")

Created word-to-index and index-to-word dictionary


In [10]:
# saving wordtoix as csv file
f_ptr = open(target_path + "wordtoix.csv", "w")
for key, value in wordtoix.items():
  f_ptr.write(str(key) + "," + str(value) + "\n")
f_ptr.close()
print("wordtoix saved as csv file")

wordtoix saved as csv file


In [11]:
# saving ixtoword as csv file
f_ptr = open(target_path + "ixtoword.csv", "w")
for key, value in ixtoword.items():
  f_ptr.write(str(key) + "," + str(value) + "\n")
f_ptr.close()
print("ixtoword saved as csv file")

ixtoword saved as csv file


In [12]:
# getting vocabulary size
vocab_size = len(wordtoix) + 1 # 1 is added for '0'
print("Vocabulary Size: ", vocab_size)

Vocabulary Size:  1652


In [13]:
# loading GloVe model
glove_model = dict()
glove_data = open(glove_model_path, encoding='utf-8')
for line in glove_data:
	values = line.split()
	word = values[0]
	feat = values[1:]
	glove_model[word] = feat
print("Loaded GloVe model")

Loaded GloVe model


In [14]:
# creating embedding matrix, i.e., glove feature for each word in vocabulary
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM)) 
for i in range(len(vocabulary)):
	embedding_vec = glove_model.get(vocabulary[i])
	if(embedding_vec is not None):
		embedding_matrix[i] = embedding_vec
print("Successfully created Embedding Matrix, i.e., GloVe bottleneck features for each word of vocabulary")

Successfully created Embedding Matrix, i.e., GloVe bottleneck features for each word of vocabulary


In [15]:
# saving embedding_matrix
savetxt(target_path + "embedding_matrix.csv", embedding_matrix, delimiter=',')
print("Successfully saved embedding_matrix.csv")

Successfully saved embedding_matrix.csv


In [16]:
# reading training file (image name and processed captions)
train_image_caption = pd.read_csv(train_image_caption_path)
print("Successfully read training - image name and processed caption file")

Successfully read training - image name and processed caption file


In [17]:
# reading training - image name and bottleneck feature file
train_image_feature = pd.read_csv(train_image_bottleneck_feature_path)
print("Sucessfully read training - image name and their InceptionV3 bottleneck feature file")

Sucessfully read training - image name and their InceptionV3 bottleneck feature file


In [18]:
# converting train_image_feature dataframe to dictionary
print("Converting training - image name and feature from dataframe to dicitonary for quick search. Wait...")
train_image_feature = train_image_feature.set_index('image').T.to_dict('list')
print("Successfully converted training - image name and feature from dataframe to dictionary")

Converting training - image name and feature from dataframe to dicitonary for quick search. Wait...
Successfully converted training - image name and feature from dataframe to dictionary


In [19]:
# steps required for training
steps = (train_image_caption.shape[0]) // NUMBER_PICS_PER_BATCH

In [20]:
# extract maximum length of caption - saved by script_preprocessing.py
f_ptr = open(max_caption_length_path, 'r')
line = f_ptr.readlines()
max_caption_length = int(line[0].split(":")[1].strip())

In [21]:
# defining model
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_caption_length,))
se1 = Embedding(vocab_size, EMBEDDING_DIM, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [22]:
# model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 28)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 28, 200)      330400      input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
______________________________________________________________________________________________

In [23]:
# use embedding matrix as weights in layer 2
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [24]:
# train network
for i in range(EPOCHS):
	generator = data_generator(train_image_caption, train_image_feature, wordtoix, max_caption_length, vocab_size, NUMBER_PICS_PER_BATCH)
	model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
	model.save(target_path + 'model_' + str(i) + '.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


2000/2000 [==============================] - 161s 80ms/step - loss: 2.3454
